# Problem: Can't Load Model (Memory Issues)

* S1: Smaller Model
* S2: https://medium.com/@giacomopedo99/running-transformer-models-without-a-gpu-overcoming-the-flash-attn-dependency-08f785675641
* S3: Simple use more memory
* Quantization
* Quantizing Models: https://www.youtube.com/watch?v=xKAXo0Zkfss

**Huh**: Why I need tokenize my prompt?

**Aha**: Maybe GPT, Gemini or when I use Pipeline this thing is happining in the Background;

**Aha**: 8bits is like a medium performance (fast and reasoble answer; 4bits is super fast bit less precise).

**Huh**: Why I'm getting so much text and not the actual answer?

**Q**: How can I load a Large Model using HuggingFace?

**Aha**: Quantization
**Huh**: Why it gets confuse when we try answer in pt-br?

## Start

In [1]:
%%capture
!pip install accelerate
!pip install -U bitsandbytes

## 4Bits-8Bits

In [2]:
%%capture
import torch
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda:0" if torch.cuda.is_available() else "cpu" 

config = BitsAndBytesConfig(load_in_8bit=True)

model_name = "Qwen/Qwen2.5-Math-7B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
    quantization_config=config
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [23]:
prompt = "What's a LLM?"

# CoT
messages = [
    {"role": "system", "content": "think step by step"},
    {"role": "user", "content": prompt}
]

# TIR
# messages = [
#     {"role": "system", "content": "Sua resposta deve ser em pt-br"},
#     {"role": "user", "content": prompt}
# ]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

In [24]:
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
response

'A LLM is a large language model created by Alibaba Cloud. It is a type of natural language processing model that can generate human-like text based on the input it receives. LLMs are used in a variety of applications, including language translation, text generation, and question answering. They are also used in games and simulations to generate realistic scenarios and outcomes.'

'A LLM is a large language model created by Alibaba Cloud. It is a type of natural language processing model that can generate human-like text based on the input it receives. LLMs are used in a variety of applications, including language translation, text generation, and question answering. They are also used in games and simulations to generate realistic scenarios and outcomes.'
